In [14]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import key_param

In [15]:
client = MongoClient(key_param.MONGO_URI)
dbName = "PKU"
collectionName = "collection_of_pku_text_blobs"
collection = client[dbName][collectionName]

ConfigurationError: The resolution lifetime expired after 21.134 seconds: Server Do53:172.20.10.1@53 answered The DNS operation timed out.; Server Do53:172.20.10.1@53 answered The DNS operation timed out.; Server Do53:172.20.10.1@53 answered The DNS operation timed out.; Server Do53:172.20.10.1@53 answered The DNS operation timed out.; Server Do53:172.20.10.1@53 answered The DNS operation timed out.; Server Do53:172.20.10.1@53 answered The DNS operation timed out.; Server Do53:172.20.10.1@53 answered The DNS operation timed out.

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=key_param.openai_api_key, disallowed_special=())

vectorStore = MongoDBAtlasVectorSearch(collection=collection, embedding=embeddings, index_name='knn')

In [35]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import key_param
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.callbacks.tracers import ConsoleCallbackHandler

client = MongoClient(key_param.MONGO_URI)
dbName = "PKU"
collectionName = "collection_of_pku_text_blobs"
collection = client[dbName][collectionName]

embeddings = OpenAIEmbeddings(openai_api_key=key_param.openai_api_key, disallowed_special=())

vectorStore = MongoDBAtlasVectorSearch(collection=collection, embedding=embeddings, index_name='knn')

template = """
Role and Expertise: You are an AI expert in inherited metabolic diseases. Your primary role is to provide comprehensive and accurate answers to questions related to inherited metabolic disorders. Your expertise is dedicated to understanding, explaining, and offering insights into these conditions.
Diagnostic Proficiency: You are proficient in analyzing diagnostic queries related to inherited metabolic diseases. When presented with symptoms, medical histories, or lab results, use your knowledge to suggest potential diagnoses, always emphasizing the need for professional medical validation.
Scope of Interaction: Focus exclusively on questions about inherited metabolic diseases. Do not engage in or provide answers to queries outside this specific domain. Your purpose is to stay within the boundaries of this medical specialty.
Response Guidelines: Ensure that your responses are informed, clear, and directly relevant to the questions asked. Be concise yet thorough in your explanations, and tailor your language to be accessible to both medical professionals and individuals without specialized medical background.
Ethical Considerations: While you can suggest potential diagnoses based on the information provided, remind users that your guidance is not a substitute for professional medical advice. Avoid making personal judgments or recommendations beyond the scope of inherited metabolic diseases.

You the following as context for your answer: {context}

Question: {question}
"""
# Don't justify your answers. Don't give information not mentioned in the CONTEXT INFORMATION. If the answer is not in the context, say the words "Sorry, I am unable to answer your question with the information available to me

def query_data(query):
    docs = vectorStore.similarity_search(query, k=1)
    as_output = docs[0].page_content

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=key_param.openai_api_key, temperature=0)
    retriever = vectorStore.as_retriever(search_kwargs={"k": 1})

    # qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)
    # retriever_output = qa.run(query)

    prompt = ChatPromptTemplate.from_template(template)

    def get_relevant_doc_string(_):
        relevent_doc = retriever.invoke(query)
        return relevent_doc[0].page_content
    
    relevent_doc = RunnablePassthrough(get_relevant_doc_string)

    setup_and_retrieval = RunnableParallel(
        {"context": relevent_doc, "question": RunnablePassthrough()}
    )
    chain = setup_and_retrieval | prompt 
    retriever_output = chain.invoke(query) # , config={'callbacks': [ConsoleCallbackHandler()]}

    return retriever_output

ro = query_data("What is PKU?")

In [36]:
ro

ChatPromptValue(messages=[HumanMessage(content='\nRole and Expertise: You are an AI expert in inherited metabolic diseases. Your primary role is to provide comprehensive and accurate answers to questions related to inherited metabolic disorders. Your expertise is dedicated to understanding, explaining, and offering insights into these conditions.\nDiagnostic Proficiency: You are proficient in analyzing diagnostic queries related to inherited metabolic diseases. When presented with symptoms, medical histories, or lab results, use your knowledge to suggest potential diagnoses, always emphasizing the need for professional medical validation.\nScope of Interaction: Focus exclusively on questions about inherited metabolic diseases. Do not engage in or provide answers to queries outside this specific domain. Your purpose is to stay within the boundaries of this medical specialty.\nResponse Guidelines: Ensure that your responses are informed, clear, and directly relevant to the questions aske

In [7]:
for item in ro:
    if item[0] == 'messages':
        messages = item[1]
        break
messages

[HumanMessage(content="\nRole and Expertise: You are an AI expert in inherited metabolic diseases. Your primary role is to provide comprehensive and accurate answers to questions related to inherited metabolic disorders. Your expertise is dedicated to understanding, explaining, and offering insights into these conditions.\nDiagnostic Proficiency: You are proficient in analyzing diagnostic queries related to inherited metabolic diseases. When presented with symptoms, medical histories, or lab results, use your knowledge to suggest potential diagnoses, always emphasizing the need for professional medical validation.\nScope of Interaction: Focus exclusively on questions about inherited metabolic diseases. Do not engage in or provide answers to queries outside this specific domain. Your purpose is to stay within the boundaries of this medical specialty.\nResponse Guidelines: Ensure that your responses are informed, clear, and directly relevant to the questions asked. Be concise yet thoroug

In [18]:
messages[0]['content']


TypeError: 'HumanMessage' object is not subscriptable

In [12]:
import re

text = str(messages[0])
pattern = r"^(.*?)(?=', metadata={'_id':')"

match = re.search(pattern, text)
if match:
    result = match.group(1)
    print(result)  # This will print everything up until the specified sequence
else:
    print("No match found")


No match found


In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=key_param.openai_api_key, temperature=0)

In [6]:
for item in ro:
    messages = item
    break

messages

('messages',
 [HumanMessage(content="\nRole and Expertise: You are an AI expert in inherited metabolic diseases. Your primary role is to provide comprehensive and accurate answers to questions related to inherited metabolic disorders. Your expertise is dedicated to understanding, explaining, and offering insights into these conditions.\nDiagnostic Proficiency: You are proficient in analyzing diagnostic queries related to inherited metabolic diseases. When presented with symptoms, medical histories, or lab results, use your knowledge to suggest potential diagnoses, always emphasizing the need for professional medical validation.\nScope of Interaction: Focus exclusively on questions about inherited metabolic diseases. Do not engage in or provide answers to queries outside this specific domain. Your purpose is to stay within the boundaries of this medical specialty.\nResponse Guidelines: Ensure that your responses are informed, clear, and directly relevant to the questions asked. Be conci

In [12]:
def query_data(query):
    docs = vectorStore.similarity_search(query, k=1)
    as_output = docs[0].page_content

    llm = OpenAI(openai_api_key=key_param.openai_api_key, temperature=0)
    retriever = vectorStore.as_retriever()
    qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)
    retriever_output = qa.run(query)

    return as_output, retriever_output

In [13]:
docs = vectorStore.similarity_search("Can you describe PKU?", k=1)
docs

[Document(page_content='Phenylketonuria (PKU) is an autosomal recessive inborn error of metabolism resulting from a deficiency of phenylalanine hydroxylase (PAH; 612349), an enzyme that catalyzes the hydroxylation of phenylalanine to tyrosine, the rate-limiting step in phenylalanine catabolism. If undiagnosed and untreated, phenylketonuria can result in impaired postnatal cognitive development resulting from a neurotoxic effect of hyperphenylalaninemia (Zurfluh et al., 2008).\n\nSee Scriver (2007) and Blau et al. (2010) for detailed reviews of PKU.', metadata={'_id': ObjectId('65a964fdae5cf51da363e067'), 'embedding': [-0.00431883228731077, 0.0026127472207069504, 0.018536207764732377, -0.03455065829848616, -0.046093541959216555, 0.02378769961216085, -0.0022845290966580015, -0.008351691711477281, -0.03330277908449613, -0.005475720347419134, 0.006648856728078941, 0.0027849806758378074, -0.010905944652236115, -0.002320275588135703, -0.019784086978722405, -0.01913414856872885, 0.02823326954